In [8]:
# pip install censusgeocode

In [42]:
import pandas as pd
import numpy as np
import censusgeocode as cg
import time
from datetime import datetime
import os

In [43]:
# Prepping the input data in chunks
        
keep_col = ['NPI','Entity Type Code','Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']
# 'Provider Last Name (Legal Name)', 'Provider First Name', 'Provider Middle Name',
# 'Provider Name Prefix Text', 'Provider Name Suffix Text',
# 'NPI Deactivation Reason Code','NPI Deactivation Date','NPI Reactivation Date',

<!-- 1. Prosthetics Case Management: 1744P3200X -->
<!-- 2. Orthotic Fitter: 225000000X -->
3. Orthotist: 222Z00000X
4. Prosthetist: 224P00000X
<!-- 5. Prosthetic/Orthotic Supplier:335E00000X -->
<!-- 6. Pedorthist: 224L00000X -->

ph_tx.to_csv('list_clinics_v1.csv',index=True)
ph_tx.to_excel('list_clinics_v1.xlsx',index=False)

In [46]:
taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
community_pharm = ['222Z00000X', '224P00000X'] #Orthotist, Prosthetist
npi_csv = 'npidata_pfile_20050523-20230212.csv' #Newer files will prob change the name

# This defines the rows I want
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'CA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    #ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta  #& ac 
    sub = data[all_together]
    return sub

def csv_chunks(file,chunk_size,keep_cols,row_sub):
    # First lets get the header and figure out the column indices
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    # Now reading in a chunk of data
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

# Takes about 3 minutes
print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)
ph_tx['Provider Business Practice Location Address Postal Code'] = ph_tx['Provider Business Practice Location Address Postal Code'].str[0:5]

ph_tx = ph_tx[['Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']]
ph_tx

2023-03-24 13:55:18.837851
Grabbed iter 1 total sub n so far 2
Grabbed iter 2 total sub n so far 7
Grabbed iter 3 total sub n so far 9
Grabbed iter 4 total sub n so far 13
Grabbed iter 5 total sub n so far 16
Grabbed iter 6 total sub n so far 25
Grabbed iter 7 total sub n so far 28
Grabbed iter 8 total sub n so far 33
2023-03-24 13:57:23.855388


,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number
0,"TERRAPIN TECHNOLOGIES, INC",1508 CABRILLO AVE,TORRANCE,CA,90501,3102123067
1,MIKE T MARTINEZ,2222 WATT AVE,SACRAMENTO,CA,95825,9164881478
2,VETERANS ADMINISTRATION PALO ALTO HEALTH CARE ...,3801 MIRANDA AVE,PALO ALTO,CA,94304,6504935000
3,RGP ORTHOPEDIC APPLIANCE CO INC,6147 UNIVERSITY AVE,SAN DIEGO,CA,92115,6195823871
4,FEELING BEAUTIFUL AGAIN,1916 NEW JERSEY STREET,FAIRFIELD,CA,94533,7073998511
5,J&K ORTHOPEDICS INC,320 E BONITA AVENUE,POMONA,CA,91767,9096211180
6,JENNINGS ORTHOPEDIC INC,10683 MAGNOLIA AVE,RIVERSIDE,CA,92505,9513522029
7,"DAW, LLC",1745 W KETTLEMAN LN,LODI,CA,95242,2094810091
8,ALPHA ORTHOPEDIC APPLIANCE CO A CORP,5940 E WASHINGTON BLVD,COMMERCE,CA,90040,3237216706
9,SOLE INNOVATIONS,10051 LAKE AVE,TRUCKEE,CA,96161,5305877461


#### Prosthetics Case Management

In [17]:
taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
community_pharm = ['1744P3200X'] #Prosthetics Case Management
npi_csv = 'npidata_pfile_20050523-20230212.csv' #Newer files will prob change the name

# This defines the rows I want
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'CA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    #ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta  #& ac 
    sub = data[all_together]
    return sub

def csv_chunks(file,chunk_size,keep_cols,row_sub):
    # First lets get the header and figure out the column indices
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    # Now reading in a chunk of data
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

# Takes about 3 minutes
print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx1 = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)
ph_tx1['Provider Business Practice Location Address Postal Code'] = ph_tx1['Provider Business Practice Location Address Postal Code'].str[0:5]

In [22]:
ph_tx1 = ph_tx1[['Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']]
ph_tx1

,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number
0,FRESNO PROSTHETIC INC,4832 N 1ST ST,FRESNO,CA,93726,5592252400
1,P V O HEALTH CARE,367 ERVILLA ST,POMONA,CA,91767,9096297615
2,MORE HAIR EXTENSIONS,4937 W SLAUSON AVE STE B2,LOS ANGELES,CA,90056,3232942548
3,MICHAEL B PHILLIPS,9253 HERMOSA AVE,RANCHO CUCAMONGA,CA,91730,9094664333
4,BEAUTIQUE SALON,1186 S 2ND ST,SAN JOSE,CA,95112,4084608070
5,CHANEL'S HAIR BOUTIQUE,8818 LA MESA BLVD,LA MESA,CA,91942,6192003186
6,MANE OF YOUR DREAM,11553 FOOTHILL BLVD STE 101,RANCHO CUCAMONGA,CA,91730,9099691833
7,HAIR AND SCALP RECOVERY CENTER,13724 BURBANK BLVD,SHERMAN OAKS,CA,91401,6618608202
8,"MISSION PROSTHETICS, INC.",568 N TUSTIN ST,ORANGE,CA,92867,7146333222
9,"MSLLL, LIMITED LIABILITY COMPANY",5650 MARCONI AVE STE 22,CARMICHAEL,CA,95608,9164108618


### Orthotic Fitter

In [29]:
taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
community_pharm = ['225000000X'] #Orthotic Fitter
npi_csv = 'npidata_pfile_20050523-20230212.csv' #Newer files will prob change the name

In [30]:
# This defines the rows I want
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'CA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    #ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta  #& ac 
    sub = data[all_together]
    return sub

In [31]:
def csv_chunks(file,chunk_size,keep_cols,row_sub):
    # First lets get the header and figure out the column indices
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    # Now reading in a chunk of data
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

In [32]:
# Takes about 3 minutes
print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

2023-03-24 13:23:09.306716
Grabbed iter 1 total sub n so far 6
Grabbed iter 2 total sub n so far 12
Grabbed iter 3 total sub n so far 16
Grabbed iter 4 total sub n so far 20
Grabbed iter 5 total sub n so far 22
Grabbed iter 6 total sub n so far 26
Grabbed iter 7 total sub n so far 28
Grabbed iter 8 total sub n so far 30
2023-03-24 13:25:11.206742


In [34]:
ph_tx2 = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)
ph_tx2['Provider Business Practice Location Address Postal Code'] = ph_tx2['Provider Business Practice Location Address Postal Code'].str[0:5]

In [35]:
ph_tx2 = ph_tx2[['Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']]
ph_tx2

,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number
0,CENTER FOR INDEPEND REHAB,1134 PLUMAS STREET,YUBA CITY,CA,95991,5308229400
1,HEALING HANDS,23586 CALABASAS ROAD,CALABASAS,CA,91302,8182243837
2,DREW HITTENBERGER AND ASSOCIATES,1111 SONOMA AVE,SANTA ROSA,CA,95405,7077651122
3,WEST COAST ORTHOTIC & PROSTHETIC SERVICES INC,3215 N CALIFORNIA ST,STOCKTON,CA,95204,2099424166
4,HEALING HANDS REHAB INC,23586 CALABASAS RD STE 206,CALABASAS,CA,91302,8182243837
5,CENTER FOR INDEPENDENT REHABILITATIVE,479 ORO DAM BLVD EAST,OROVILLE,CA,95965,5305349500
6,HAND HEALTH RESOURCES INC,2323 DE LA VINA ST,SANTA BARBARA,CA,93105,8056823055
7,STOCKTON HAND THERAPY & REHABILITATION,1919 GRAND CANAL BLVD,STOCKTON,CA,95207,2099568737
8,MEDCESSITY INC.,4010 ORANGE AVE,LONG BEACH,CA,90807,5624283556
9,US SPINE & SPORT KEARNY VILLA,3444 KEARNY VILLA RD,SAN DIEGO,CA,92123,8585739368


### Pedorthist

In [36]:
taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
community_pharm = ['224L00000X'] #Pedorthist
npi_csv = 'npidata_pfile_20050523-20230212.csv' #Newer files will prob change the name

# This defines the rows I want
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'CA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    #ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta  #& ac 
    sub = data[all_together]
    return sub

def csv_chunks(file,chunk_size,keep_cols,row_sub):
    # First lets get the header and figure out the column indices
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    # Now reading in a chunk of data
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

# Takes about 3 minutes
print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx3 = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)
ph_tx3['Provider Business Practice Location Address Postal Code'] = ph_tx3['Provider Business Practice Location Address Postal Code'].str[0:5]

ph_tx3 = ph_tx3[['Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']]
ph_tx3

2023-03-24 13:34:07.674217
Grabbed iter 1 total sub n so far 0
Grabbed iter 2 total sub n so far 0
Grabbed iter 3 total sub n so far 0
Grabbed iter 4 total sub n so far 1
Grabbed iter 5 total sub n so far 3
Grabbed iter 6 total sub n so far 3
Grabbed iter 7 total sub n so far 4
Grabbed iter 8 total sub n so far 4
2023-03-24 13:36:11.708011


,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number
0,VITO'S PEDORTHIC CENTER,143 E ROWLAND ST,COVINA,CA,91723,6268589460
1,WEST COAST DME & SUPPLIES LLC,8322 CLAIREMONT MESA BLVD STE 105,SAN DIEGO,CA,92111,9094773117
2,JENNIFER FARIA,301 GEORGIA ST STE 322,VALLEJO,CA,94590,5106726694
3,VITOS PEDORTHIC CENTER INC,143 E ROWLAND ST STE 2,COVINA,CA,91723,6268589460


### Prosthetic/Orthotic Supplier

In [55]:
taxon_codes = ['Healthcare Provider Taxonomy Code_' + str(i+1) for i in range(15)]
keep_col += taxon_codes
community_pharm = ['335E00000X'] #Prosthetic/Orthotic Supplier
npi_csv = 'npidata_pfile_20050523-20230212.csv' #Newer files will prob change the name

# This defines the rows I want
def sub_rows(data):
    ec = data['Entity Type Code'] == "2"
    st = data['Provider Business Practice Location Address State Name'] == 'CA'
    ta = data[taxon_codes].isin(community_pharm).any(axis=1)
    #ac = data['NPI Deactivation Reason Code'].isna()
    all_together = ec & st & ta  #& ac 
    sub = data[all_together]
    return sub

def csv_chunks(file,chunk_size,keep_cols,row_sub):
    # First lets get the header and figure out the column indices
    header_fields = list(pd.read_csv(npi_csv, nrows=1))
    header_locs = [header_fields.index(i) for i in keep_cols]
    # Now reading in a chunk of data
    skip = 1
    it_n = 0
    sub_n = 0
    ret_chunk = chunk_size
    fin_li_dat = []
    while ret_chunk == chunk_size:
        file_chunk = pd.read_csv(file, usecols=header_locs, skiprows=skip, 
                     nrows=chunk_size, names=header_fields, dtype='str')
        sub_dat = row_sub(file_chunk)
        fin_li_dat.append( sub_dat.copy() )
        skip += chunk_size
        it_n += 1
        sub_n += sub_dat.shape[0]
        print(f'Grabbed iter {it_n} total sub n so far {sub_n}')
        ret_chunk = file_chunk.shape[0]
    fin_dat = pd.concat(fin_li_dat, axis=0)
    return fin_dat

# Takes about 3 minutes
print( datetime.now() )
pharm_tx = csv_chunks(npi_csv, chunk_size=1000000, keep_cols=keep_col, row_sub=sub_rows)
print( datetime.now() )

ph_tx4 = pharm_tx.drop(columns=taxon_codes).reset_index(drop=True)
ph_tx4['Provider Business Practice Location Address Postal Code'] = ph_tx4['Provider Business Practice Location Address Postal Code'].str[0:5]

ph_tx4 = ph_tx4[['Provider Organization Name (Legal Business Name)',
            'Provider First Line Business Practice Location Address',
            'Provider Business Practice Location Address City Name',
            'Provider Business Practice Location Address State Name',
            'Provider Business Practice Location Address Postal Code',
            'Provider Business Practice Location Address Telephone Number']]
ph_tx4

2023-03-24 14:02:08.290469
Grabbed iter 1 total sub n so far 65
Grabbed iter 2 total sub n so far 181
Grabbed iter 3 total sub n so far 331
Grabbed iter 4 total sub n so far 429
Grabbed iter 5 total sub n so far 507
Grabbed iter 6 total sub n so far 635
Grabbed iter 7 total sub n so far 731
Grabbed iter 8 total sub n so far 810
2023-03-24 14:04:13.637012


,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number
0,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,UNION CITY,CA,94587,5103243400
1,"SOUND BALANCE AUDIOLOGY, INC",2420 VISTA WAY,OCEANSIDE,CA,92054,7607217417
2,COLLIER REHABILITATION SYSTEMS,3161 PUTNAM BLVD,PLEASANT HILL,CA,94523,9259431119
3,"NEW DAY'S DAWN, INC.",10159 MISSION GORGE RD,SANTEE,CA,92071,6195964042
4,SUPER CARE INC,16017 VALLEY BLVD,CITY OF INDUSTRY,CA,91744,6268542283
...,...,...,...,...,...,...
805,"ACTIVE LIFE, LLC",18433 ROSCOE BLVD STE 108,NORTHRIDGE,CA,91325,8188359441
806,"ACTIVE LIFE, LLC",16008 KAMANA RD STE 202A,APPLE VALLEY,CA,92307,7605156311
807,"ACTIVE LIFE, LLC",7910 FROST ST STE 320,SAN DIEGO,CA,92123,6194886196
808,"ACTIVE LIFE, LLC",1700 E CESAR E CHAVEZ AVE STE 3750,LOS ANGELES,CA,90033,3233528319


In [90]:
ph_ph = ph_tx4[ph_tx4['Provider Organization Name (Legal Business Name)'].str.contains("PROSTHETICS") & ph_tx4['Provider Organization Name (Legal Business Name)'].str.contains("ORTHOTICS")]

In [91]:
ph_ph

,Provider Organization Name (Legal Business Name),Provider First Line Business Practice Location Address,Provider Business Practice Location Address City Name,Provider Business Practice Location Address State Name,Provider Business Practice Location Address Postal Code,Provider Business Practice Location Address Telephone Number
0,DIMENSION PROSTHETICS & ORTHOTICS,33374 DOWE AVE,UNION CITY,CA,94587,5103243400
5,"HANGER PROSTHETICS & ORTHOTICS WEST, INC.",3460 ROBIN LN,CAMERON PARK,CA,95682,5306764856
8,HANGER PROSTHETICS & ORTHOTICS WEST INC,7056 A SKYWAY,PARADISE,CA,95969,5308771805
24,"BAKERSFIELD PROSTHETICS & ORTHOTICS CENTER, INC.",2023 TRUXTUN AVE,BAKERSFIELD,CA,93301,6612812127
28,"VENTURA PROSTHETICS& ORTHOTICS, INC.",2324 N BATAVIA ST,ORANGE,CA,92865,7143573940
...,...,...,...,...,...,...
787,HANGER PROSTHETICS & ORTHOTICS WEST INC,4849 LONE TREE WAY STE A,ANTIOCH,CA,94531,9257541804
789,"ALLIANCE PROSTHETICS & ORTHOTICS, INC.",12458 WASHINGTON BLVD,WHITTIER,CA,90602,5629451111
790,AVOP-ANTELOPE VALLEY ORTHOTICS & PROSTHETICS,525 COMMERCE AVE,PALMDALE,CA,93551,6612670772
792,"HANGER PROSTHETICS & ORTHOTICS WEST, INC.",3155 KEARNEY ST STE 130,FREMONT,CA,94538,5104906400


In [89]:
# ph_tx4

In [88]:
# ph_tx4[ph_tx4['Provider Organization Name (Legal Business Name)'].str.contains("LIMB")]

In [94]:
result = pd.concat([ph_tx, ph_ph])

In [95]:
result.to_csv('list_clinics_v1.csv',index=True)
result.to_excel('list_clinics_v1.xlsx',index=False)